In [1]:
from sklearn.preprocessing import MinMaxScaler

/home/yurko/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import numpy as np
import pandas as pd

In [3]:
mm_scaler = MinMaxScaler().set_output(transform="pandas")

In [4]:
df_data = pd.read_csv("raw_data/AAPL.csv")

In [5]:
df_macroeconomic = pd.read_csv("raw_data/macro_economic_indicators.csv")

In [6]:
df_technical = pd.read_csv("raw_data/AAPL_technical_analysis.csv")

# Supporting functions

In [7]:
def rename_macroeconomic(df_macroeconomic):
    columns_macroeconomic = ["datetime", "interest_rate", "GDP", "inflation"]
    df_macroeconomic.columns = columns_macroeconomic
    return df_macroeconomic

In [8]:
def merge_macroeconomic(df_technical, df_macroeconomic):
    df_merged = df_technical.merge(df_macroeconomic, on="datetime", how="inner")
    return df_merged

In [9]:
def rename_columns(df):
    columns = ["datetime", "open", "high", "low", "close", "volume"]
    df.columns = columns
    return df

In [10]:
def rename_technical(df_technical):
    df_technical = df_technical.rename(columns = {"Unnamed: 0": "datetime"})
    return df_technical

In [11]:
def convert_datetime(df):
    data = df.copy()
    data["datetime"] = pd.to_datetime(data["datetime"])
    return data

In [12]:
def merge_columns(df_values, df_technical):
    df_merged = df_values.merge(df_technical, how="inner", on="datetime")
    return df_merged

In [13]:
def clean_data(df):
    df_inter = df.interpolate(method='linear')
    df_clean = df_inter.dropna()
    return df_clean

In [14]:
def create_target(df, column_name, new_column_name='Target'):   
    result = []
    for i in range(len(df) - 1):
        if df[column_name].iloc[i] > df[column_name].iloc[i-1]:
            result.append(1)
        else:
            result.append(0)
    result.append(float('nan'))
    df[new_column_name] = pd.Series(result, index=df.index)
    return df

In [15]:
def target_drop(df_final):
    df_final = df_final.dropna()
    return df_final

In [16]:
def convert_time_sin_cos(df):
    data = df.copy()
    data['day_of_year'] = data['datetime'].dt.dayofyear
    data['time_of_day'] = data['datetime'].dt.hour * 3600 + data['datetime'].dt.minute * 60 + data['datetime'].dt.second
    data['day_of_year_norm'] = data['day_of_year'] / 365.0
    data['time_of_day_norm'] = data['time_of_day'] / 86400.0
    data['cos_time_of_day'] = np.cos(2 * np.pi * data['time_of_day_norm'])
    data['sin_time_of_day'] = np.sin(2 * np.pi * data['time_of_day_norm'])
    data["cos_day_of_year"] = np.cos(2 * np.pi * data['day_of_year_norm'])
    data["sin_day_of_year"] = np.sin(2 * np.pi * data['day_of_year_norm'])
    df_converted = data.drop(columns = ["day_of_year", "time_of_day", "day_of_year_norm", "time_of_day_norm", "datetime"])
    return df_converted

In [17]:
def log_divide_next(df):
    log_returns = np.log(df_data['close'] / df_data['close'].shift(1))
    return log_returns

# Final function that prepares all the data

In [18]:
def create_x_y(df_data, df_technical, df_macroeconomic):
    df_macroeconomic = rename_macroeconomic(df_macroeconomic)
    df_technical = rename_technical(df_technical)
    df_data = rename_columns(df_data)
    df_technical = clean_data(df_technical)
    df_data = convert_datetime(df_data)
    df_technical = convert_datetime(df_technical)
    df_macroeconomic = convert_datetime(df_macroeconomic)
    df_technical = merge_macroeconomic(df_technical, df_macroeconomic)
    df_merged = merge_columns(df_data, df_technical)
    df_target = create_target(df_merged, "close")
    df_final = target_drop(df_target)
    df_time = convert_time_sin_cos(df_target)
    X = df_time.drop(columns = "Target")
    y = df_time[["Target"]]
    log_df = log_divide_next(df_data["close"]).dropna()
    return X, y, log_df

In [19]:
X, y, log_df = create_x_y(df_data, df_technical, df_macroeconomic)

In [21]:
def create_lstm_windows(X, y, window_size):
    
    X_windows = []
    y_windows = []
    
    mm_scaler = MinMaxScaler().set_output(transform="pandas")
    X_scaled = mm_scaler.fit_transform(X)
    for i in range(len(X) - window_size):
        X_windows.append(X.iloc[i:i + window_size].values)
        y_windows.append(y.iloc[i + window_size])
    
    return np.array(X_windows), np.array(y_windows)


In [22]:
X_windows, y_windows = create_lstm_windows(X, y, 10)

In [28]:
from tensorflow.keras import models
from tensorflow.keras import Sequential, layers
from tensorflow.keras import optimizers
from tensorflow.keras import callbacks
from tensorflow.keras.layers import Dense, Reshape
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Precision, Accuracy

In [24]:
model = Sequential()
model.add(layers.LSTM(units = 128, return_sequences = True, input_shape = (X_windows.shape[1:])))
model.add(layers.Dropout(0.2))
model.add(layers.LSTM(units = 64, return_sequences = True))
model.add(layers.Dropout(0.2))
model.add(layers.LSTM(units = 64, return_sequences = True))
model.add(layers.Dropout(0.2))
model.add(layers.LSTM(units = 32, return_sequences = True))
model.add(layers.Dropout(0.2))
model.add(layers.LSTM(units = 8))
model.add(layers.Dense(1, activation="sigmoid"))

I0000 00:00:1724836712.972016  192497 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-28 10:18:32.972299: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/yurko/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [25]:
es = EarlyStopping(patience=3, restore_best_weights=True)

In [30]:
model.compile(loss="binary_crossentropy", optimizer = "adam", metrics = ["accuracy", "precision"])

In [31]:
history = model.fit(X_windows, y_windows, epochs = 10, batch_size = 64)

Epoch 1/5
  85/2177 ━━━━━━━━━━━━━━━━━━━━ 36s 18ms/step - accuracy: 0.5073 - loss: 0.6937 - precision: 0.5055

KeyboardInterrupt: 